In [1]:
:set -XNoImplicitPrelude
:set -XOverloadedStrings
:set -XLambdaCase

In [2]:
import Papa
import Control.Monad.State
import Control.Monad.State.Class
import qualified Prelude as P (read, undefined)
import qualified Data.IntMap as IM
import Data.Monoid
import Control.Monad.Reader



In [3]:
type Input = [Int]
type Output = IM.IntMap Int

data ParseResult' a =  Halt' Output | Result' Input a | Unknown' deriving (Show)

instance Functor ParseResult' where
    _ `fmap` Halt' a = Halt' a
    _ `fmap` Unknown' = Unknown'
    f `fmap` Result' i a = Result' i (f a)


isHalt' :: ParseResult' a -> Bool
isHalt' (Halt' _) = True
isHalt' _ = False

isError :: ParseResult' a -> Bool
isError Unknown' = True
isError _ = False

onResult :: ParseResult' a -> 
    (Input -> a -> ParseResult' b) -> 
    ParseResult' b
onResult Unknown' _ = Unknown'
onResult (Halt' i) _ = Halt' i
onResult (Result' i a) k = k i a

-- onHalt :: ParseResult' a -> Maybe Output
-- onHalt (Halt' i) = Just i
-- onHalt _ = Nothing

newtype Parser' a = Parser' (Input -> ParseResult' a) 

parse' :: Parser' a -> Input -> ParseResult' a
parse' (Parser' p) = p

instance Functor Parser' where
    f `fmap` (Parser' p) = Parser' (fmap f . p )

instance Applicative Parser' where
    pure a = Parser' (`Result'` a) -- valueParser
    
    (Parser' f) <*> (Parser' p) = Parser' (\i -> case p i of
        Halt' a -> Halt' a
        Unknown' -> Unknown'
        Result' j a -> (\g -> g a) <$> f j)
        
instance Monad Parser' where
    (Parser' p) >>= f = Parser' (\i ->
        onResult (p i) (\j a -> parse' (f a) j))
        
(|||) :: Parser' a -> Parser' a -> Parser' a
(Parser' p1) ||| (Parser' p2) = Parser' (\i -> let v = p1 i in bool v (p2 i) (isError v || isHalt' v))

list1 :: Parser' a -> Parser' [a]
list1 p = p >>= (\a -> (a :) <$> list p)

list :: Parser' a -> Parser' [a]
list p = list1 p ||| pure []

        
addParser' :: Parser' (State Output ())
addParser' = Parser' (
    \case
        l@(o : a : b : c : xs) | o == 1 ->
            Result' xs (modify (\s -> 
                IM.update 
                    ((liftA2 (+) (s IM.!? a) (s IM.!? b) <|>) . Just) 
                    c
                    s
                ))
        _ -> Unknown'
    )
        
prodParser' :: Parser' (State Output ())
prodParser' = Parser' (
    \case
        l@(o : a : b : c : xs) | o == 2 ->
            Result' xs (modify (\s -> 
                IM.update 
                    ((liftA2 (*) (s IM.!? a) (s IM.!? b) <|>) . Just) 
                    c
                    s
                ))
        _ -> Unknown'
    )
    
    
haltParser' :: Parser' (State Output ())
haltParser' = Parser' (
    \case
        l@(o : _) | o == 99 -> Halt' (IM.fromList $ itoList l)
        _ -> Unknown')
        

        
extract :: Input -> ParseResult' [State Output ()] -> Output
extract i p =  let ii = IM.fromList $ itoList i in 
    case fmap (foldl' (flip execState) ii) p of
        Result' _ a -> a
        _ -> IM.empty

In [4]:
let d = [1,0,0,3,1,1,2,3,1,3,4,3,1,5,0,3,2,1,10,19,1,19,6,23,2,23,13,27,1,27,5,31,2,31,10,35,1,9,35,39,1,39,9,43,2,9,43,47,1,5,47,51,2,13,51,55,1,55,9,59,2,6,59,63,1,63,5,67,1,10,67,71,1,71,10,75,2,75,13,79,2,79,13,83,1,5,83,87,1,87,6,91,2,91,13,95,1,5,95,99,1,99,2,103,1,103,6,0,99,2,14,0,0]
let dd = d & ix 1 .~ 12 & ix 2 .~ 2

let resParser = parse' (list (prodParser' ||| addParser' ||| haltParser'))

liftA2 (IM.lookup 0 .) extract resParser dd

Just 3790645

In [5]:
let anvs = [(noun,verb)| noun <- [0..99], verb <- [0..99]]

let mkInp n v = d & ix 1 .~ n & ix 2 .~ v

f (a : n : v : _) | a == 19690720 = True
f _  = False

g (_ : n : v : _) = Just $ 100 * n + v
g _ = Nothing


g =<< (find f $ 
            (liftA2 (toListOf (taking 3 traverse) .) extract resParser) 
            <$> 
            (uncurry mkInp <$> anvs))

Just 6577